# Installing the NAG Library and running this notebook
To run this notebook, you will need to install the NAG Library for Python (Mark 29.3 or newer) and a license key. You can find the software and have a license key (trials are available) from our website here: [Getting Started with the NAG Library](https://www.nag.com/content/getting-started-nag-library?lang=py&os=linux)

We are solving a classic portfolio optimization problem using the NAG Library integration in CVXPY. It can be formulated in the following way:

\begin{equation*}
    \begin{aligned}
        &\min_{x \in \mathbb{R}^n} &&\frac{1}{2} x^T Q_0 x + r^T_0 x
        \\
        &\textrm{subject to } &&\frac{1}{2} x^T Q_1 x + r^T_1 x \leq 0,
        \\
        & &&e^Tx = 1,
        \\
        & && x \geq 0,
    \end{aligned}
\end{equation*}
where $e$ refers to the vector of all ones.

In [1]:
# Import modules
from cvxpy import Minimize, Problem, Variable, quad_form, sum, atoms, OPTIMAL, NAG
import numpy as np
import naginterfaces

In [2]:
# Define number of assets
n = 500 

# Generate random data
np.random.seed(2)
r0 = np.matrix(np.random.randn(n, 1))
r1 = np.matrix(np.random.randn(n, 1))
q0 = np.matrix(np.random.randn(n, n))
q0 = q0.T * q0
q1 = np.matrix(np.random.randn(n, n))
q1 = q1.T * q1

# Skip psd check due to issues with ARPACK
q0 = atoms.affine.wraps.psd_wrap(q0)
q1 = atoms.affine.wraps.psd_wrap(q1)

# Create the cvxpy problem:
# Define the variables
x = Variable(len(r1))

# Define the constraints
constraints = [
               0 <= x, 
               sum(x) == 1,
               0.5*quad_form(x, q1) + (r1.T @ x) <= 0]

# Define the objective function
objective = Minimize(0.5*quad_form(x, q0) + r0.T @ x)

# Set up dictionary for option setting
nag_params = {'SOCP System Formulation':'AS',
              'SOCP Factorization Method':'MA86'
              }

# Define the problem
prob = Problem(objective, constraints)

# Solve the problem using NAG
prob.solve(solver=NAG, nag_params=nag_params)

-0.17415932352686342